#####           v0 06/06/2022
#####           SCIENCE TEAM BRAZIL
#####           THE NATURE CONSERVANCY

##### FONTE DO DADO: https://alerta.mapbiomas.org/

##### JUPYTER NOTEBOOK PARA AUTOMATIZAR A VERIFICAÇÃO DE DESMATAMENTOS COMPILADOS PELO MAPBIOMAS ALERTA, NA BASE TNC (ACELERADOR, CACAU FLORESTA E SIR)


In [ ]:
#imports
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.offsetbox import AnchoredText

#pip installations
#!pip install pygeos

In [ ]:
#create mapbiomas alert geopandas
#load mapbiomas alerta layer (initially, using shapefile downloaded in official website
zipfile = r"C:\Users\guilherme.fronza\OneDrive\temp\20220603_mapbiomas_alerta\dashboard_alerts-shapefile.zip"
mapbiomas_alert = gpd.read_file(zipfile, encoding='utf-8')
#crs='epsg:4674' # Sirgas2000 DMS
mapbiomas_alert = mapbiomas_alert.to_crs(4674)

In [ ]:
mapbiomas_alert

In [ ]:
def processing_mapbiomas_alert(shp_path, epsg, mapbiomas_alert_layer):
    #albers equal area projstring - IBGE reference
    ibge_projstring = '+proj=aea +lat_1=-2 +lat_2=-22 +lat_0=-12 +lon_0=-54 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'
    
    #tnc database preparation
    tnc_database = gpd.read_file(shp_path, encoding='utf-8')
    tnc_database = tnc_database.to_crs(epsg)
    
    #first intersection - tnc database alerts
    alerts_found = gpd.sjoin(tnc_database, mapbiomas_alert_layer, how='inner', predicate='intersects')
    #alerts_found = alerts_found.reset_index()
    
    #second intersection - mapbiomas layers alerts (to use on plot)
    alerts_found_mapbiomas_layer = gpd.sjoin(mapbiomas_alert_layer, tnc_database, how='inner', predicate='intersects')
    #alerts_found_mapbiomas_layer = alerts_found_mapbiomas_layer.reset_index()

    #third intersection - results - polygons to calculate area in hectars - Albers Equal Area calc
    result_intersection_layer = mapbiomas_alert_layer.overlay(tnc_database, how='intersection')
    result_intersection_layer = result_intersection_layer.to_crs(ibge_projstring)
    result_intersection_layer["area_ha"] = result_intersection_layer['geometry'].area/10000
    result_intersection_layer = result_intersection_layer.to_crs(4674)
    result_intersection_layer = result_intersection_layer.reset_index()

    return alerts_found, alerts_found_mapbiomas_layer, result_intersection_layer


def plot_mapbiomas_alert(df_alert, df_mapbiomas_alerts_layer, df_result_intersect, project_name, mapbiomas_legend_text, tnc_legend_text, date_ext, id_prop_name):
    for index, row in df_result_intersect.iterrows():
        print(index)
        #create figure
        figure, ax = plt.subplots(figsize=(15,10), dpi=200)
        #set title
        ax.set_title('Projeto: '+ project_name + ' | '+ 'Polígonos identificados com interseção nos dados Mapbiomas Alerta - Extração ' + date_ext, 
                     fontdict={'fontsize': 8, 'fontweight': 'medium'})
        #geopandas plot per intersect
        df_result_intersect.iloc[[index]].plot(ax=ax, color='magenta', edgecolor='magenta', linewidth=2)
        
        df_alert.plot(ax=ax, color='none', edgecolor='yellow', linewidth=1)
        
        df_mapbiomas_alerts_layer.plot(ax=ax, color='none', edgecolor='red', linewidth=1)
        
        df_result_intersect['geometry_buffer'] = df_result_intersect.geometry.buffer(0.02)
        #min max settings
        minx, miny, maxx, maxy = df_result_intersect.iloc[[index]].geometry_buffer.total_bounds
        ax.set_xlim(minx, maxx)
        ax.set_ylim(miny, maxy)
        #basemap plot
        cx.add_basemap(ax, crs= df_result_intersect.crs.to_string(), source = cx.providers.Esri.WorldImagery)
        #area alerta
        alert_area = str(df_result_intersect.area_ha.iloc[index].round(2))
        #legend build
        mapbiomas_legend = Line2D([0], [0], label=mapbiomas_legend_text, color='red')
        tnc_database_legend = Line2D([0], [0], label=tnc_legend_text, color='yellow')
        intersection_tnc_legenda = Line2D([0], [0], label=str('Área de interseção: ' + alert_area + ' ha') , color='magenta')
        ax.legend(handles=[mapbiomas_legend,tnc_database_legend,intersection_tnc_legenda])
        
        #codigos de identificação
        id_prop = str('Produtor: ' + df_result_intersect[id_prop_name].iloc[index])
        data_detec = str('Data Detec.: '+ df_result_intersect.DataDetec.iloc[index])
        vpressao = str('Pressão : '+ df_result_intersect.VPressao.iloc[index])
        
        # text anchor
        at = AnchoredText(
            {id_prop, data_detec, vpressao}, prop=dict(size=8), frameon=True, loc='lower left')
        at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
        ax.add_artist(at)
        #plt.text(0.5, 0.5, 'Some text', transform=ax.transAxes)
        plt.savefig(df_result_intersect[id_prop_name].iloc[index] + '_' + date_ext + '_idx'+ str(index+1)+'.png', dpi=200)
        plt.close(figure)

In [ ]:
# #RUN AMAZON ACELERATOR
alerts_found_ARA, alerts_found_mapbiomas_layer_ARA, result_intersection_layer_ARA = processing_mapbiomas_alert(shp_path=r"C:\Users\guilherme.fronza\OneDrive\temp\202203_acelerador_dbase\ippara_ara_base_de_dados_v20220428_IR.shp",
        epsg=4674, mapbiomas_alert_layer=mapbiomas_alert)

In [ ]:
#export to xlsx
result_intersection_layer_ARA.to_excel(r'result_intersection_layer_ARA.xlsx', index = False)

In [ ]:
#PLOT AMAZON ACELERATOR ALERTS    
plot_mapbiomas_alert(df_alert=alerts_found_ARA, 
                 df_mapbiomas_alerts_layer=alerts_found_mapbiomas_layer_ARA,
                 df_result_intersect=result_intersection_layer_ARA,
                 project_name= 'Acelerador TNC', 
                 mapbiomas_legend_text='Mapbiomas Alerta - polígonos de desmatamento',
                 tnc_legend_text='Base de dados TNC', 
                 date_ext= '2022607'
                 id_prop_name='Name')

In [ ]:
#RUN COCOA FOREST PROJECT 
alerts_found_CF, alerts_found_mapbiomas_layer_CF, result_intersection_layer_CF = processing_mapbiomas_alert(shp_path=r"C:\Users\guilherme.fronza\OneDrive\temp\202203_cacau_floresta_dbase\pa_cacauflor_propriedades_202203.shp",
        epsg=4674, mapbiomas_alert_layer=mapbiomas_alert)

In [ ]:
#export to xlsx
result_intersection_layer_CF.to_excel(r'result_intersection_layer_CF.xlsx', index = False)

In [ ]:
#PLOT COCOA FOREST ALERTS        
plot_mapbiomas_alert(df_alert=alerts_found_CF, 
                 df_mapbiomas_alerts_layer=alerts_found_mapbiomas_layer_CF,
                 df_result_intersect=result_intersection_layer_CF,
                 project_name= 'COCOA FOREST', 
                 mapbiomas_legend_text='Mapbiomas Alerta - polígonos de desmatamento',
                 tnc_legend_text='Base de dados TNC', 
                 date_ext= '2022607',
                 id_prop_name='Name')

In [ ]:
#RUN SIR POLYGONS
alerts_found_SIR, alerts_found_mapbiomas_layer_SIR, result_intersection_layer_SIR = processing_mapbiomas_alert(shp_path=r"C:\Users\guilherme.fronza\OneDrive\sir\sir_202104\shp_download.shp",
        epsg=4674, mapbiomas_alert_layer=mapbiomas_alert)

In [ ]:
#export to xlsx
result_intersection_layer_SIR.to_excel(r'result_intersection_layer_SIR.xlsx', index = False)

In [ ]:
#PLOT SIR ALERTS     
plot_mapbiomas_alert(df_alert=alerts_found_SIR, 
                 df_mapbiomas_alerts_layer=alerts_found_mapbiomas_layer_SIR,
                 df_result_intersect=result_intersection_layer_SIR,
                 project_name= 'SIR TNC', 
                 mapbiomas_legend_text='Mapbiomas Alerta - polígonos de desmatamento',
                 tnc_legend_text='Base de dados TNC', 
                 date_ext= '2022607',
                 id_prop_name='PROPRIETAR')